<a href="https://colab.research.google.com/github/vivekellanti/pothole-detectionYOLOv8/blob/main/pothole_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q ultralytics
!pip install -q kaggle
!pip install -q seaborn
!pip install -q matplotlib
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.6/779.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 54.1 MB/s eta 0:00:00


In [2]:
!pip install -q yolov9

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.4/953.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.6/112.6 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 

In [4]:
from google.colab import drive
from ultralytics import YOLO
import seaborn as sns
import matplotlib.pyplot as plt
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!mkdir -p ~/.kaggle
!echo '{"username":"vivekellanti","key":"157397c0f9f4206a129a69bb9d43984a"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json



In [7]:
!kaggle datasets download -d rajdalsaniya/pothole-detection-dataset
!yes | unzip -q /content/drive/MyDrive/pothole-detection-dataset.zip -d /content/drive/MyDrive/pothole-detection-dataset


Dataset URL: https://www.kaggle.com/datasets/rajdalsaniya/pothole-detection-dataset
License(s): CC0-1.0
pothole-detection-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
unzip:  cannot find or open /content/drive/MyDrive/pothole-detection-dataset.zip, /content/drive/MyDrive/pothole-detection-dataset.zip.zip or /content/drive/MyDrive/pothole-detection-dataset.zip.ZIP.


In [11]:
import os

# Define the dataset path
dataset_path = '/content/drive/MyDrive/pothole-detection-dataset'

# List of required directories
required_dirs = [
    os.path.join(dataset_path, 'train/images'),
    os.path.join(dataset_path, 'train/labels'),
    os.path.join(dataset_path, 'valid/images'),
    os.path.join(dataset_path, 'valid/labels'),
    os.path.join(dataset_path, 'test/images'),
    os.path.join(dataset_path, 'test/labels')
]

# Check and create missing directories
for dir in required_dirs:
    if not os.path.exists(dir):
        os.makedirs(dir)
        print(f"Created missing directory: {dir}")

# Verify paths in data.yaml
data_yaml_path = os.path.join(dataset_path, 'data.yaml')
with open(data_yaml_path, 'w') as f:
    f.write(f"""
train: {os.path.join(dataset_path, 'train/images')}
val: {os.path.join(dataset_path, 'valid/images')}
test: {os.path.join(dataset_path, 'test/images')}

nc: 1  # Number of classes
names: ['pothole']  # Class names
""")

print("Updated data.yaml with correct paths.")


Updated data.yaml with correct paths.


In [12]:
import os

# Define the dataset path
dataset_path = '/content/drive/MyDrive/pothole-detection-dataset'

# Check the validation images directory
val_images_path = os.path.join(dataset_path, 'valid/images')
if os.path.exists(val_images_path):
    print(f"Validation images directory exists: {val_images_path}")
    print("Listing files in the directory:")
    val_images = os.listdir(val_images_path)
    print(val_images)
    if not val_images:
        print("No images found in the validation directory.")
else:
    print(f"Validation images directory does NOT exist: {val_images_path}")

Validation images directory exists: /content/drive/MyDrive/pothole-detection-dataset/valid/images
Listing files in the directory:
['553_jpg.rf.a5586080c55d3f70e84a9f6e4264acc8.jpg', '595_jpg.rf.aabfba06a8e77786d7c7ea84ab85aa4b.jpg', '552_jpg.rf.b38e499d872c1ed47b181022b8b1ee9d.jpg', '8_jpg.rf.88508100d861f1957cb2b3b231ac9ef1.jpg', '552_jpg.rf.47c9daa4474beee1ebaaa8979a76abae.jpg', '9_jpg.rf.0fb30cfe880b3e1510466e6c20575ba3.jpg', '596_jpg.rf.897113e5a3ee905d3afad5340386aa0b.jpg', '13_jpg.rf.4dd24cd41caf2edb92bade1c834cf2b8.jpg', '13_jpg.rf.34885bb42e4ba1f9116aa9e124c95f82.jpg', '8_jpg.rf.1c96f62d936a4c9f0d6a49bce97bf010.jpg', '55_jpg.rf.400e2e551363b197413591d43b898663.jpg', '243_jpg.rf.fe58ca02b741c3fe23dc011b812d00a0.jpg', '596_jpg.rf.d935a38d5bf91568757605ad470fd3ce.jpg', '595_jpg.rf.bdce8cee8f9d1283851517aa8de79a30.jpg', '.ipynb_checkpoints']


In [13]:
from ultralytics import YOLO


#Load yolov8 model
model = YOLO('yolov8n.pt')

#Train model using data.yaml file
model.train(data='/content/drive/MyDrive/pothole-detection-dataset/data.yaml',
            epochs=50,
            imgsz=640,
            batch=16,
            weight_decay=0.0005,
            momentum=0.9,
            optimizer='Adam',
            patience=10,
            augment=True,
            save_period=10,
            verbose=False)
#Validate model on test data and get predictions
results = model.val(data = '/content/drive/MyDrive/pothole-detection-dataset/data.yaml', split = 'test')



Ultralytics YOLOv8.2.28 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/pothole-detection-dataset/data.yaml, epochs=50, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=10, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=Adam, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_la

train: Scanning /content/drive/MyDrive/pothole-detection-dataset/train/labels.cache... 2067 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2067/2067 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/drive/MyDrive/pothole-detection-dataset/valid/labels... 13 images, 1 backgrounds, 0 corrupt: 100%|██████████| 14/14 [00:32<00:00,  2.31s/it]

val: New cache created: /content/drive/MyDrive/pothole-detection-dataset/valid/labels.cache


...

val: Scanning /content/drive/MyDrive/pothole-detection-dataset/test/labels... 1 images, 1 backgrounds, 0 corrupt: 100%|██████████| 2/2 [00:01<00:00,  1.49it/s]

val: New cache created: /content/drive/MyDrive/pothole-detection-dataset/test/labels.cache



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.33it/s]


                   all          2          1      0.468          1      0.497      0.249
Speed: 0.4ms preprocess, 126.9ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/train22


TypeError: 'list' object is not callable

In [1]:
for result in results:
    print(result.keys())

NameError: name 'results' is not defined